# Финальное задание (Иванов Павел)

## 1. Работа с данными

#### Импортируем нужные библиотеки

In [62]:
import numpy as np
import pandas as pd
import datetime
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

#### Считываем данные

In [63]:
data_x = pd.read_csv('C:/features.csv', index_col='match_id')
data_x.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


#### Выделяем целевую переменную "radiant_win"

In [64]:
data_y = data_x.pop('radiant_win')

#### Удалим признаки сильно связанные с целевой меткой, что бы алгоритмы сильно на них не настраивались

In [65]:
data_x.pop('tower_status_radiant') 
data_x.pop('tower_status_dire') 
data_x.pop('barracks_status_radiant') 
data_x.pop('barracks_status_dire')

match_id
0          0
1          1
2         63
3         63
4         63
5         63
8         63
9          0
11        63
12        63
14        63
15        63
17         0
20        15
21        63
22         0
23        48
25         3
26         0
27        48
28         3
29         3
30         3
31         0
32         3
34         0
35        63
36        63
38        63
39        63
          ..
114373    51
114374    63
114375    51
114376    63
114379     3
114380    63
114381    63
114382    59
114383    63
114384     3
114385    63
114386    51
114387     3
114388     3
114389     0
114390    63
114391    63
114392    63
114394    56
114395     0
114396     2
114397     0
114399    63
114400     0
114401    35
114402    63
114403     3
114404    63
114405    63
114406     0
Name: barracks_status_dire, Length: 97230, dtype: int64

#### Удалим непонятный признак

In [66]:
data_x.pop('duration') 
data_x.shape

(97230, 102)

#### Найдём пропуски в данных

In [67]:
data_x.columns[list(set(np.where(pd.isnull(data_x))[1]))]

Index(['dire_flying_courier_time', 'dire_first_ward_time', 'first_blood_time',
       'first_blood_team', 'first_blood_player1', 'first_blood_player2',
       'radiant_bottle_time', 'radiant_courier_time',
       'radiant_flying_courier_time', 'radiant_first_ward_time',
       'dire_bottle_time', 'dire_courier_time'],
      dtype='object')

Получаем 12 признаков с пропусками:
#1  'dire_flying_courier_time'- время приобретения предмета "flying_courier", вероятно герой не использовал его;
#2  'dire_first_ward_time' - время установки командой первого "наблюдателя", возможно команда не успела сделать это;
#3  'first_blood_time' -  игровое время первой крови, возможно в первые 5 минут не было этого события;
#4  'first_blood_team' - команда, совершившая первую кровь, возможно в первые 5 минут не было этого события; 
#5  'first_blood_player1' - игрок, причастный к событию, похоже нет события - нет метки об игроке; 
#6  'first_blood_player2' - второй игрок, причастный к событию, похоже нет события - нет метки об игроке;
#7  'radiant_bottle_time' - время первого приобретения командой предмета "bottle", возможно предмет не приобретался; 
#8  'radiant_courier_time' - время приобретения предмета "courier", возможно предмет не приобретался во время снятия представленных данных;
#9 'radiant_flying_courier_time'- время приобретения предмета "flying_courier", вероятно герой не использовал его; 
#10 'radiant_first_ward_time' - время установки командой первого "наблюдателя", возможно команда не успела сделать это;  
#11 'dire_bottle_time' - время первого приобретения командой предмета "bottle", возможно предмет не приобретался; 
#12 'dire_courier_time' - время приобретения предмета "courier", возможно предмет не приобретался во время снятия представленных данных.


#### Заменим пропуски

In [68]:
data_x = data_x.fillna(0)
data_x.columns[list(set(np.where(pd.isnull(data_x))[1]))] #Ожидаем массива без объектов

Index([], dtype='object')

#### Настрои кросс-валидацию

In [8]:
cv = KFold(n_splits=5, shuffle=True, random_state=241);

## 2. Градиентный бустинг

#### Запускаем обучение

In [9]:
for i in (10, 20, 30, 40, 50):
    print ('Градиентный бустинг:', i, 'деревьев')
    score = 0;
    model1 = GradientBoostingClassifier(n_estimators=i, random_state=241)
    start_time = datetime.datetime.now()
    for train, test in cv.split(data_x):
        model1.fit(data_x.iloc[train], data_y.iloc[train])
        predict = model1.predict_proba(data_x.iloc[test])
        score = score + roc_auc_score(data_y.iloc[test], predict.transpose()[1])
    print ('ROC-AUC:', score/5)
    print ('Алгоритм обучался по KFold:', datetime.datetime.now() - start_time)

Градиентный бустинг: 10 деревьев
ROC-AUC: 0.6636054026335821
Алгоритм обучался по KFold: 0:00:45.487909
Градиентный бустинг: 20 деревьев
ROC-AUC: 0.6870475739899333
Алгоритм обучался по KFold: 0:01:16.082897
Градиентный бустинг: 30 деревьев
ROC-AUC: 0.6975609299728723
Алгоритм обучался по KFold: 0:02:13.435672
Градиентный бустинг: 40 деревьев
ROC-AUC: 0.7036277765401293
Алгоритм обучался по KFold: 0:03:10.761452
Градиентный бустинг: 50 деревьев
ROC-AUC: 0.7080931711048563
Алгоритм обучался по KFold: 0:03:49.806635


## 3. Линейная регрессия

#### Нормализуем данные

In [69]:
scaler = StandardScaler()
data_x_normal = scaler.fit_transform(data_x)

#### Запускаем обучение

In [11]:
for i in (0.00001, 0.0001, 0.001, 0.01, 1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0):
    print ('Регуляризация(C):', i)
    score = 0;
    model2 = LogisticRegression(random_state=241, penalty='l2', C=i)
    start_time = datetime.datetime.now()
    for train, test in cv.split(data_x_normal):
        model2.fit(data_x_normal[train], data_y.iloc[train])
        predict = model2.predict_proba(data_x_normal[test])
        score = score + roc_auc_score(data_y.iloc[test], predict.transpose()[1])
    print ('ROC-AUC:', score/5)
    print ('Алгоритм обучался по KFold:', datetime.datetime.now() - start_time)

Регуляризация(C): 1e-05
ROC-AUC: 0.6975163523542433
Алгоритм обучался по KFold: 0:00:05.925687
Регуляризация(C): 0.0001
ROC-AUC: 0.714059039904311
Алгоритм обучался по KFold: 0:00:09.290335
Регуляризация(C): 0.001
ROC-AUC: 0.7189976467269782
Алгоритм обучался по KFold: 0:00:17.713959
Регуляризация(C): 0.01
ROC-AUC: 0.7191670463622615
Алгоритм обучался по KFold: 0:00:22.888095
Регуляризация(C): 1.0
ROC-AUC: 0.71913613970466
Алгоритм обучался по KFold: 0:00:25.372010
Регуляризация(C): 10.0
ROC-AUC: 0.7191355252996446
Алгоритм обучался по KFold: 0:00:25.430042
Регуляризация(C): 100.0
ROC-AUC: 0.7191353706280719
Алгоритм обучался по KFold: 0:00:25.737378
Регуляризация(C): 1000.0
ROC-AUC: 0.7191353791073956
Алгоритм обучался по KFold: 0:00:26.047168
Регуляризация(C): 10000.0
ROC-AUC: 0.719135376987021
Алгоритм обучался по KFold: 0:00:24.987703
Регуляризация(C): 100000.0
ROC-AUC: 0.7191353812240984
Алгоритм обучался по KFold: 0:00:25.539126


## 4. Линейная регрессия - категориальные признаки

#### Удалим категориальные признаки

In [70]:
data_x_without = data_x.copy()
print('Размер объекта до:', data_x_without.shape)
data_x_without.pop('lobby_type')
data_x_without.pop('r1_hero')
data_x_without.pop('r2_hero')
data_x_without.pop('r3_hero')
data_x_without.pop('r4_hero')
data_x_without.pop('r5_hero')
data_x_without.pop('d1_hero')
data_x_without.pop('d2_hero')
data_x_without.pop('d3_hero')
data_x_without.pop('d4_hero')
data_x_without.pop('d5_hero')
print('Размер объекта после:', data_x_without.shape)

Размер объекта до: (97230, 102)
Размер объекта после: (97230, 91)


#### Нормализуем данные

In [71]:
scaler = StandardScaler()
data_x_without_normal = scaler.fit_transform(data_x_without)

#### Запускаем обучение

In [14]:
for i in (0.00001, 0.0001, 0.001, 0.01, 1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0):
    print ('Регуляризация(C):', i)
    score = 0;
    model3 = LogisticRegression(random_state=241, penalty='l2', C=i)
    start_time = datetime.datetime.now()
    for train, test in cv.split(data_x_without_normal):
        model3.fit(data_x_without_normal[train], data_y.iloc[train])
        predict = model3.predict_proba(data_x_without_normal[test])
        score = score + roc_auc_score(data_y.iloc[test], predict.transpose()[1])
    print ('ROC-AUC:', score/5)
    print ('Алгоритм обучался по KFold:', datetime.datetime.now() - start_time)

Регуляризация(C): 1e-05
ROC-AUC: 0.6974806717615767
Алгоритм обучался по KFold: 0:00:04.888385
Регуляризация(C): 0.0001
ROC-AUC: 0.7140607212347175
Алгоритм обучался по KFold: 0:00:08.014314
Регуляризация(C): 0.001
ROC-AUC: 0.7190274774316969
Алгоритм обучался по KFold: 0:00:15.102961
Регуляризация(C): 0.01
ROC-AUC: 0.7191907150638098
Алгоритм обучался по KFold: 0:00:21.389856
Регуляризация(C): 1.0
ROC-AUC: 0.7191575272634679
Алгоритм обучался по KFold: 0:00:23.064175
Регуляризация(C): 10.0
ROC-AUC: 0.7191573449191618
Алгоритм обучался по KFold: 0:00:23.008132
Регуляризация(C): 100.0
ROC-AUC: 0.7191573406634937
Алгоритм обучался по KFold: 0:00:23.078791
Регуляризация(C): 1000.0
ROC-AUC: 0.7191573618462848
Алгоритм обучался по KFold: 0:00:23.388540
Регуляризация(C): 10000.0
ROC-AUC: 0.7191573512446267
Алгоритм обучался по KFold: 0:00:23.179300
Регуляризация(C): 100000.0
ROC-AUC: 0.7191573491249009
Алгоритм обучался по KFold: 0:00:22.515709


## 5. Линейная регрессия + мешок слов

#### Определение уникальных героев

In [15]:
heros_id = set(data_x[['r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']].values.ravel())
count = 0
for i in heros_id:
    count = count + 1
print ('Количество уникальных героев в НАШЕЙ выборке:', count)

Количество уникальных героев в НАШЕЙ выборке: 108


#### Добавление мешка слов

In [72]:
X_pick = np.zeros((data_x.shape[0], 112)) #Используем максимальный id героя в выборке
print ('Размер до мешка слов:', data_x_without_normal.shape)
for i, match_id in enumerate(data_x.index):
    for p in (0, 1, 2, 3, 4):
        X_pick[i, data_x.ix[match_id, 'r%d_hero' % (p+1)] - 1] = 1
        X_pick[i, data_x.ix[match_id, 'd%d_hero' % (p+1)] - 1] = -1
data_x_withBag_normal = np.column_stack((data_x_without_normal, X_pick))
print ('Размер с мешком слов:', data_x_withBag_normal.shape)

Размер до мешка слов: (97230, 91)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


Размер с мешком слов: (97230, 203)


#### Запускаем обучение 

In [18]:
for i in (0.00001, 0.0001, 0.001, 0.01, 1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0):
    print ('Регуляризация(C):', i)
    score = 0;
    model4 = LogisticRegression(random_state=241, penalty='l2', C=i)
    start_time = datetime.datetime.now()
    for train, test in cv.split(data_x_withBag_normal):
        model4.fit(data_x_withBag_normal[train], data_y.iloc[train])
        predict = model4.predict_proba(data_x_withBag_normal[test])
        score = score + roc_auc_score(data_y.iloc[test], predict.transpose()[1])
    print ('ROC-AUC:', score/5)
    print ('Алгоритм обучался по KFold:', datetime.datetime.now() - start_time)

Регуляризация(C): 1e-05
ROC-AUC: 0.7015617104785626
Алгоритм обучался по KFold: 0:00:05.632555
Регуляризация(C): 0.0001
ROC-AUC: 0.7275936120136889
Алгоритм обучался по KFold: 0:00:09.114179
Регуляризация(C): 0.001
ROC-AUC: 0.748555854643419
Алгоритм обучался по KFold: 0:00:20.552200
Регуляризация(C): 0.01
ROC-AUC: 0.7539132525548672
Алгоритм обучался по KFold: 0:00:33.174699
Регуляризация(C): 1.0
ROC-AUC: 0.7540937106329834
Алгоритм обучался по KFold: 0:00:48.642505
Регуляризация(C): 10.0
ROC-AUC: 0.754091723690034
Алгоритм обучался по KFold: 0:00:47.673569
Регуляризация(C): 100.0
ROC-AUC: 0.7540914651488226
Алгоритм обучался по KFold: 0:00:48.854014
Регуляризация(C): 1000.0
ROC-AUC: 0.754091437607695
Алгоритм обучался по KFold: 0:00:48.330088
Регуляризация(C): 10000.0
ROC-AUC: 0.7540913804366418
Алгоритм обучался по KFold: 0:00:47.625585
Регуляризация(C): 100000.0
ROC-AUC: 0.7540913041249849
Алгоритм обучался по KFold: 0:00:46.597724


## 6. Обучение финальной модели

In [73]:
data_test = pd.read_csv('C:/features_test.csv', index_col='match_id')
data_test = data_test.fillna(0)
data_test_without = data_test.copy()

In [74]:
print('Размер объекта до:', data_test_without.shape)
data_test_without.pop('lobby_type')
data_test_without.pop('r1_hero')
data_test_without.pop('r2_hero')
data_test_without.pop('r3_hero')
data_test_without.pop('r4_hero')
data_test_without.pop('r5_hero')
data_test_without.pop('d1_hero')
data_test_without.pop('d2_hero')
data_test_without.pop('d3_hero')
data_test_without.pop('d4_hero')
data_test_without.pop('d5_hero')
print('Размер объекта после:', data_test_without.shape)

Размер объекта до: (17177, 102)
Размер объекта после: (17177, 91)


In [75]:
scaler = StandardScaler()
data_test_without_normal = scaler.fit_transform(data_test_without)

In [76]:
X_pick = np.zeros((data_test.shape[0], 112)) #Используем максимальный id героя в выборке
print ('Размер до мешка слов:', data_test_without.shape)
for i, match_id in enumerate(data_test.index):
    for p in (0, 1, 2, 3, 4):
        X_pick[i, data_test.ix[match_id, 'r%d_hero' % (p+1)] - 1] = 1
        X_pick[i, data_test.ix[match_id, 'd%d_hero' % (p+1)] - 1] = -1
data_test_withBag = np.column_stack((data_test_without_normal, X_pick))
print ('Размер с мешком слов:', data_test_withBag.shape)

Размер до мешка слов: (17177, 91)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


Размер с мешком слов: (17177, 203)


In [77]:
model5 = LogisticRegression(random_state=241, penalty='l2', C=100000)

In [78]:
model5.fit(data_x_withBag_normal, data_y)

LogisticRegression(C=100000, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=241,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [83]:
predict = model5.predict_proba(data_test_withBag)

In [84]:
print(predict)

[[0.17420978 0.82579022]
 [0.24122368 0.75877632]
 [0.81303031 0.18696969]
 ...
 [0.7652392  0.2347608 ]
 [0.37113753 0.62886247]
 [0.57456705 0.42543295]]
